# Lab 6: MAF Dev UI - Integrated Workflow Example

## Overview

Experience both **Sequential → Concurrent** patterns in a single workflow.

**Workflow Structure:**
```
Planner (Sequential) → Broadcast → (Culture || Food || Nature) Parallel → Combiner → Finalizer
```

---


## 1. Environment Setup


In [ ]:
# Import libraries
import json
import threading
from dataclasses import dataclass

from azure.identity import (
    AzureCliCredential,
    ChainedTokenCredential,
    ManagedIdentityCredential,
)
from azure.identity.aio import (
    AzureCliCredential as AsyncAzureCliCredential,
    ChainedTokenCredential as AsyncChainedTokenCredential,
    ManagedIdentityCredential as AsyncManagedIdentityCredential,
)

from agent_framework import WorkflowBuilder, WorkflowContext, executor
from agent_framework.azure import AzureAIAgentClient
from agent_framework.devui import serve

print("✅ Libraries loaded successfully")


In [ ]:
# Load config
with open('config.json', 'r') as f:
    config = json.load(f)

PROJECT_CONNECTION_STRING = config['project_connection_string']
MODEL_DEPLOYMENT_NAME = config.get('model_deployment_name', 'gpt-4o')

endpoint = PROJECT_CONNECTION_STRING.split(';')[0]

print(f"✅ Endpoint: {endpoint}")
print(f"   Model: {MODEL_DEPLOYMENT_NAME}")


In [ ]:
# Initialize Agent Client (with Tracing enabled)
from azure.ai.projects.aio import AIProjectClient

async_credential = AsyncChainedTokenCredential(
    AsyncManagedIdentityCredential(),
    AsyncAzureCliCredential()
)

# Parse Connection String
# Format: "endpoint;subscription_id;resource_group;project_name"
parts = PROJECT_CONNECTION_STRING.split(';')
subscription_id = parts[1] if len(parts) > 1 else None
resource_group = parts[2] if len(parts) > 2 else None
project_name = parts[3] if len(parts) > 3 else None

# AI Project Client for Tracing
project_client = AIProjectClient(
    endpoint=endpoint,
    subscription_id=subscription_id,
    resource_group_name=resource_group,
    project_name=project_name,
    credential=async_credential
)

agent_client = AzureAIAgentClient(
    project_endpoint=endpoint,
    model_deployment_name=MODEL_DEPLOYMENT_NAME,
    async_credential=async_credential,
    project_client=project_client  # Enable Tracing
)

# Create Agent
travel_agent = agent_client.create_agent(
    name="TravelAgent",
    instructions="Travel information expert. Answer concisely."
)

print("✅ Agent initialized successfully")


## 2. Define Integrated Workflow


In [ ]:
# Define context
@dataclass
class TravelWorkflowContext(WorkflowContext):
    destination: str = ""
    initial_plan: str = ""
    culture_info: str = ""
    food_info: str = ""
    nature_info: str = ""
    final_guide: str = ""

print("✅ Context definition complete")


In [ ]:
# Define nodes

# Sequential node
@executor(id="planner")
async def planner_node(context: TravelWorkflowContext, ctx: WorkflowContext) -> None:
    """Step 1: Draft plan creation (sequential)"""
    print(f"\n📝 [Planner] Creating travel plan for {context.destination}...")
    
    thread = travel_agent.get_new_thread()
    result = await travel_agent.run(
        f"Write a brief overview of a 2-night 3-day trip to {context.destination}.",
        thread=thread
    )
    context.initial_plan = result.text if hasattr(result, 'text') else str(result)
    
    print("✅ [Planner] Complete")
    await ctx.send_message(context, target_id="broadcast")

# Broadcast node
@executor(id="broadcast")
async def broadcast_node(context: TravelWorkflowContext, ctx: WorkflowContext) -> None:
    """Step 2: Start parallel analysis"""
    print("\n📢 [Broadcast] Starting parallel analysis...")
    await ctx.send_message(context, target_id="culture")
    await ctx.send_message(context, target_id="food")
    await ctx.send_message(context, target_id="nature")
    print("✅ [Broadcast] Complete")

# Concurrent nodes
@executor(id="culture")
async def culture_node(context: TravelWorkflowContext, ctx: WorkflowContext) -> None:
    """Step 3-1: Culture analysis (parallel)"""
    print("🏛️ [Culture] Analyzing...")
    thread = travel_agent.get_new_thread()
    result = await travel_agent.run(
        f"Recommend 3 major cultural/historical sites in {context.destination}",
        thread=thread
    )
    context.culture_info = result.text if hasattr(result, 'text') else str(result)
    await ctx.send_message(context, target_id="combiner")
    print("✅ [Culture] Complete")

@executor(id="food")
async def food_node(context: TravelWorkflowContext, ctx: WorkflowContext) -> None:
    """Step 3-2: Food analysis (parallel)"""
    print("🍜 [Food] Analyzing...")
    thread = travel_agent.get_new_thread()
    result = await travel_agent.run(
        f"Recommend 3 signature dishes in {context.destination}",
        thread=thread
    )
    context.food_info = result.text if hasattr(result, 'text') else str(result)
    await ctx.send_message(context, target_id="combiner")
    print("✅ [Food] Complete")

@executor(id="nature")
async def nature_node(context: TravelWorkflowContext, ctx: WorkflowContext) -> None:
    """Step 3-3: Nature analysis (parallel)"""
    print("🌿 [Nature] Analyzing...")
    thread = travel_agent.get_new_thread()
    result = await travel_agent.run(
        f"Recommend 3 natural scenic spots in {context.destination}",
        thread=thread
    )
    context.nature_info = result.text if hasattr(result, 'text') else str(result)
    await ctx.send_message(context, target_id="combiner")
    print("✅ [Nature] Complete")

# Combiner node
@executor(id="combiner")
async def combiner_node(context: TravelWorkflowContext, ctx: WorkflowContext) -> None:
    """Step 4: Combine results"""
    print("\n🔗 [Combiner] Combining results...")
    context.final_guide = f"""
# {context.destination} Travel Guide

## Overview
{context.initial_plan}

## 🏛️ Culture & History
{context.culture_info}

## 🍜 Food
{context.food_info}

## 🌿 Nature
{context.nature_info}
"""
    print("✅ [Combiner] Complete")
    await ctx.send_message(context, target_id="finalizer")

# Finalizer node
@executor(id="finalizer")
async def finalizer_node(context: TravelWorkflowContext, ctx: WorkflowContext) -> None:
    """Step 5: Final output"""
    print("\n✨ [Finalizer] Final processing...")
    await ctx.yield_output(context)
    print("✅ [Finalizer] Complete")

print("✅ Node definitions complete")

In [ ]:
# Build workflow
travel_workflow = (
    WorkflowBuilder(name="Travel Guide Workflow")
    # Sequential part
    .set_start_executor(planner_node)
    .add_edge(planner_node, broadcast_node)
    # Concurrent part (Fan-out)
    .add_edge(broadcast_node, culture_node)
    .add_edge(broadcast_node, food_node)
    .add_edge(broadcast_node, nature_node)
    # Concurrent → Sequential (Fan-in)
    .add_edge(culture_node, combiner_node)
    .add_edge(food_node, combiner_node)
    .add_edge(nature_node, combiner_node)
    # Final step
    .add_edge(combiner_node, finalizer_node)
    .build()
)

print("✅ Workflow build complete")
print("   Planner → Broadcast → (Culture || Food || Nature) → Combiner → Finalizer")


## 3. Start Dev UI Server

> **⚠️ If working on GitHub Codespaces:**
> 
> You need to change `host='127.0.0.1'` to `host='0.0.0.0'` in the cell below.
> 
> **Reason:**
> - `127.0.0.1`: Only accessible from localhost (only accessible from within the local machine)
> - `0.0.0.0`: Accessible from all network interfaces (accessible from outside)
> - GitHub Codespaces is a remote container environment, so the port must be exposed externally to access from the browser.
> - Codespaces automatically provides port forwarding, but the server must listen on `0.0.0.0` for port forwarding to work.

In [ ]:
# Start Dev UI server
import socket

def is_port_in_use(port):
    """Check if port is in use"""
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        return s.connect_ex(('127.0.0.1', port)) == 0

def start_devui():
    print("="*70)
    print("🌐 Starting Dev UI server...")
    print("="*70)
    print("\n🔗 Open http://localhost:8080 in your browser\n")
    
    # ⚠️ If working on GitHub Codespaces:
    # Change host='127.0.0.1' → host='0.0.0.0'
    serve(
        entities=[travel_workflow],
        port=8080,
        host='127.0.0.1',  # Codespaces: change to '0.0.0.0'
        auto_open=False,
        ui_enabled=True,
        tracing_enabled=True  # ✅ Enable Tracing
    )

# Check if port is in use
if is_port_in_use(8080):
    print("⚠️  Port 8080 is already in use.")
    print("   Dev UI server is already running or another process is using it.")
    print("\n✅ Use existing server: http://localhost:8080")
    print("\n💡 To restart the server:")
    print("   1. Restart Jupyter Notebook kernel (Kernel > Restart)")
    print("   2. Or in terminal: lsof -ti:8080 | xargs kill -9")
else:
    # Run in background thread
    server_thread = threading.Thread(target=start_devui, daemon=True)
    server_thread.start()

    import time
    time.sleep(2)

    print("✅ Dev UI server is running!")
    print("   http://localhost:8080")
    print("\n💡 For GitHub Codespaces users:")
    print("   1. Click on the 'Forwarded Address' URL for port 8080 in the PORTS tab")
    print("   2. Or click 'Open in Browser' in the popup notification")
    print("\n💡 For local environment usage:")
    print("   1. Open the URL above in your browser")
    print("   2. Click the 'Run' button")
    print("   3. Input: {\"destination\": \"Jeju Island\"}")
    print("   4. Monitor workflow execution!")

### Workflow Structure Visualization

Below is the workflow structure you can see in the Dev UI:

![Travel Guide Workflow](attachments/travel_guide_workflow.png)

The workflow executes as follows:
- **Planner**: Establish initial travel plan
- **Broadcast**: Start parallel analysis tasks
- **Culture, Food, Nature**: Perform analysis for each topic simultaneously
- **Combiner**: Integrate all analysis results into one
- **Finalizer**: Generate final travel guide

#### Information Available in Dev UI

When you run the agent in Dev UI, you can check the following information in real-time:

- **Workflow Diagram**: Visually track the currently executing node through the diagram above
- **Events**: Track execution events and state changes for each node
- **Traces**: View complete tracing information and debugging logs of agent execution
- **Execution Results**: Check input/output data for each node

This allows you to monitor and debug the agent's behavior.

## 4. Stop Dev UI Server


In [ ]:
# Stop Dev UI server
print("🛑 To stop the Dev UI server, choose one of the following methods:\n")
print("Method 1: Restart Jupyter Notebook kernel")
print("   - Menu: Kernel > Restart Kernel")
print("   - Shortcut: Command/Ctrl + Shift + P → 'Jupyter: Restart Kernel'\n")
print("Method 2: Manual shutdown in terminal")
print("   Run the cell below to copy the terminal command and execute it\n")
print("=" * 70)

In [ ]:
# Terminal command (copy and run in terminal)
print("📋 Copy the command below and run it in terminal:\n")
print("lsof -ti:8080 | xargs kill -15")
print("\n💡 If force shutdown is needed:")
print("lsof -ti:8080 | xargs kill -9")

## 4. (Optional) Run Directly in Notebook

**⚠️ We recommend running in Dev UI!**

In [ ]:
# Run workflow in notebook (for reference)
async def run_workflow():
    context = TravelWorkflowContext(destination="Busan")
    
    print("🎯 Starting workflow execution...\n")
    
    outputs = []
    async for event in travel_workflow.run_stream(context):
        if hasattr(event, 'output') and event.output is not None:
            outputs.append(event.output)
    
    result = outputs[-1] if outputs else context
    
    print("\n" + "="*70)
    print("✅ Execution complete!")
    print("="*70)
    print(result.final_guide)
    
    return result

# Execute
# result = await run_workflow()

print("💡 Uncomment the above cell to execute")
print("   or run in Dev UI!")

## 📍 Next Steps

You've learned how to use MAF Dev UI! Now move on to the final step to evaluate the agent:

1. **Notebook 07**: Agent Evaluation (`07_evaluate_agents.ipynb`)